In [3]:
import numpy as np
from collections import defaultdict, Counter
import math
from typing import List, Dict, Tuple, Set, Optional, Any
import networkx as nx
import matplotlib.pyplot as plt

class POSGraph:
    """
    Implements a graph-based structure for POS sequence processing and predictive coding.
    Each node represents a POS tag, and edges represent transitions with weights as probabilities.
    """
    
    def __init__(self, predefined_boundaries: Optional[Dict[Tuple[str, str], float]] = None):
        """
        Initialize the POS graph.
        
        Args:
            predefined_boundaries: Optional dictionary of predefined boundary probabilities
                                  for POS tag transitions (pos1, pos2) -> boundary_probability
        """
        # Main transition graph
        self.graph = nx.DiGraph()
        
        # Higher-order chunk graph for learned patterns
        self.chunk_graph = nx.DiGraph()
        
        # Track n-gram counts for training
        self.unigram_counts = Counter()
        self.bigram_counts = defaultdict(Counter)
        self.trigram_counts = defaultdict(lambda: defaultdict(Counter))
        
        # Boundary probabilities
        self.boundary_probs = defaultdict(float)
        
        # Predefined linguistic rules
        self.predefined_boundaries = predefined_boundaries or {
            ('NOUN', 'VERB'): 0.9,      # NP to VP transition
            ('VERB', 'DET'): 0.8,       # VP to NP transition
            ('PUNCT', 'DET'): 0.95,     # Punctuation followed by determiner
            ('NOUN', 'PREP'): 0.7,      # NP to PP transition
            ('VERB', 'PREP'): 0.6,      # VP to PP transition
            ('ADJ', 'NOUN'): 0.2,       # Within NP (low boundary probability)
            ('DET', 'ADJ'): 0.1,        # Within NP (very low boundary probability)
        }
        
        # Thresholds
        self.hard_boundary_threshold = 0.75
        self.soft_boundary_threshold = 0.4
        
        # Discovered chunks
        self.common_chunks = {}
        
        # Add special start and end nodes
        self.graph.add_node("<START>", pos_type="special")
        self.graph.add_node("<END>", pos_type="special")

    def train(self, pos_sequences: List[List[str]]):
        """
        Train the POS graph on a corpus of POS tag sequences.
        
        Args:
            pos_sequences: List of POS tag sequences, each representing a sentence
        """
        # 1. Collect n-gram statistics and build basic graph
        self._build_initial_graph(pos_sequences)
        
        # 2. Calculate edge weights (transition probabilities)
        self._calculate_edge_weights()
        
        # 3. Calculate boundary probabilities
        self._calculate_boundary_probabilities()
        
        # 4. Identify common chunks
        self._identify_common_chunks(pos_sequences)
        
        # 5. Build higher-order chunk graph
        self._build_chunk_graph()
        
        print(f"Training complete on {len(pos_sequences)} sentences.")
        print(f"POS graph has {len(self.graph.nodes)} nodes and {len(self.graph.edges)} edges")
        print(f"Chunk graph has {len(self.chunk_graph.nodes)} nodes and {len(self.chunk_graph.edges)} edges")

    def _build_initial_graph(self, pos_sequences: List[List[str]]):
        """Build the initial graph structure and collect statistics."""
        # First pass - add all nodes and count statistics
        for sequence in pos_sequences:
            # Add nodes for each unique POS tag
            for pos in sequence:
                if not self.graph.has_node(pos):
                    self.graph.add_node(pos, pos_type="basic")
                self.unigram_counts[pos] += 1
            
            # Count bigrams and add edges
            for i in range(len(sequence) - 1):
                pos1, pos2 = sequence[i], sequence[i+1]
                self.bigram_counts[pos1][pos2] += 1
                
                # Ensure edge exists (weight will be calculated later)
                if not self.graph.has_edge(pos1, pos2):
                    self.graph.add_edge(pos1, pos2, weight=0, count=0, boundary_prob=0)
                
                # Increment edge count
                self.graph[pos1][pos2]["count"] += 1
            
            # Add connections from start and to end
            if sequence:
                if not self.graph.has_edge("<START>", sequence[0]):
                    self.graph.add_edge("<START>", sequence[0], weight=0, count=0, boundary_prob=0)
                self.graph["<START>"][sequence[0]]["count"] += 1
                
                if not self.graph.has_edge(sequence[-1], "<END>"):
                    self.graph.add_edge(sequence[-1], "<END>", weight=0, count=0, boundary_prob=0)
                self.graph[sequence[-1]]["<END>"]["count"] += 1
            
            # Count trigrams
            for i in range(len(sequence) - 2):
                pos1, pos2, pos3 = sequence[i], sequence[i+1], sequence[i+2]
                self.trigram_counts[pos1][pos2][pos3] += 1

    def _calculate_edge_weights(self):
        """Calculate edge weights (transition probabilities) based on counts."""
        # For each node, calculate outgoing transition probabilities
        for node in self.graph.nodes():
            if node == "<END>":
                continue  # End node has no outgoing edges
                
            # Get total count of outgoing transitions
            outgoing_edges = list(self.graph.out_edges(node, data=True))
            total_count = sum(data["count"] for _, _, data in outgoing_edges)
            
            if total_count > 0:
                # Calculate probability for each outgoing edge
                for _, target, data in outgoing_edges:
                    prob = data["count"] / total_count
                    self.graph[node][target]["weight"] = prob

    def _calculate_boundary_probabilities(self):
        """Calculate boundary probabilities for each edge based on transition statistics."""
        for source, target, data in self.graph.edges(data=True):
            if source in ("<START>", "<END>") or target in ("<START>", "<END>"):
                continue  # Skip special nodes
                
            # Calculate surprisal for this transition
            prob = data["weight"]
            if prob > 0:
                surprisal = -math.log2(prob)
                
                # Normalize surprisal to a boundary probability between 0 and 1
                # Higher surprisal = higher boundary probability
                boundary_prob = 1 / (1 + math.exp(-(surprisal - 1)))
                
                # Consider predefined boundaries if available
                if (source, target) in self.predefined_boundaries:
                    predefined_prob = self.predefined_boundaries[(source, target)]
                    alpha = 0.3  # Weight for predefined rules
                    boundary_prob = alpha * predefined_prob + (1 - alpha) * boundary_prob
                
                # Store in graph and in lookup dictionary
                self.graph[source][target]["boundary_prob"] = boundary_prob
                self.boundary_probs[(source, target)] = boundary_prob

    def _identify_common_chunks(self, pos_sequences: List[List[str]]):
        """Identify common chunks based on frequency and boundary probabilities."""
        # Use a sliding window approach to find potential chunks
        chunk_candidates = Counter()
        
        # Try different chunk sizes
        for size in range(2, 5):  # 2-grams to 4-grams
            for sequence in pos_sequences:
                if len(sequence) < size:
                    continue
                    
                for i in range(len(sequence) - size + 1):
                    chunk = tuple(sequence[i:i+size])
                    chunk_candidates[chunk] += 1
        
        print(f"Found {len(chunk_candidates)} potential chunks")
        
        # For small training sets, lower the threshold to ensure we find some chunks
        total_sentences = len(pos_sequences)
        min_occurrences = max(2, int(total_sentences * 0.05))  # At least 2 occurrences or 5% of sentences
        
        # Lower the cohesion threshold for small datasets
        cohesion_threshold = 0.6 if total_sentences < 20 else 0.7
        
        # Count the chunks that meet our criteria
        qualifying_chunks = 0
        for chunk, count in chunk_candidates.items():
            if count >= min_occurrences:
                qualifying_chunks += 1
                # Calculate internal cohesion (inverse of average boundary probability)
                internal_boundaries = 0
                for i in range(len(chunk) - 1):
                    pos1, pos2 = chunk[i], chunk[i+1]
                    if (pos1, pos2) in self.boundary_probs:
                        internal_boundaries += self.boundary_probs[(pos1, pos2)]
                    else:
                        internal_boundaries += 0.5  # Default if unseen
                
                avg_internal_boundary = internal_boundaries / (len(chunk) - 1)
                cohesion = 1 - avg_internal_boundary
                
                # Only keep reasonably cohesive chunks
                if cohesion > cohesion_threshold:
                    chunk_name = f"{'_'.join(chunk)}"
                    self.common_chunks[chunk] = {
                        "name": chunk_name,
                        "elements": chunk,
                        "count": count,
                        "cohesion": cohesion,
                        "activation": 0.0  # Initial activation level
                    }
                    
        print(f"{qualifying_chunks} chunks met frequency criteria, {len(self.common_chunks)} met cohesion criteria")

    def _build_chunk_graph(self):
        """Build higher-order graph representing transitions between chunks."""
        # Add nodes for each chunk
        for chunk_tuple, chunk_info in self.common_chunks.items():
            chunk_name = chunk_info["name"]
            self.chunk_graph.add_node(
                chunk_name, 
                pos_type="chunk", 
                elements=chunk_info["elements"],
                cohesion=chunk_info["cohesion"]
            )
        
        # Connect chunks that can follow each other
        for chunk1_tuple, chunk1_info in self.common_chunks.items():
            for chunk2_tuple, chunk2_info in self.common_chunks.items():
                # Check if chunk2 can follow chunk1 (overlap or adjacency)
                if self._can_follow(chunk1_tuple, chunk2_tuple):
                    # Calculate transition probability
                    # This is simplified - would need corpus analysis for accurate probabilities
                    transition_prob = 0.1  # Default low probability
                    
                    # If we have trigram data, use it to estimate transition probability
                    if len(chunk1_tuple) >= 2 and len(chunk2_tuple) >= 1:
                        last1, last2 = chunk1_tuple[-2], chunk1_tuple[-1]
                        first = chunk2_tuple[0]
                        
                        if last2 in self.trigram_counts.get(last1, {}):
                            total = sum(self.trigram_counts[last1][last2].values())
                            if total > 0:
                                count = self.trigram_counts[last1][last2].get(first, 0)
                                transition_prob = count / total
                    
                    # Add edge with weight
                    chunk1_name = chunk1_info["name"]
                    chunk2_name = chunk2_info["name"]
                    self.chunk_graph.add_edge(
                        chunk1_name, 
                        chunk2_name, 
                        weight=transition_prob
                    )

    def _can_follow(self, chunk1: Tuple[str, ...], chunk2: Tuple[str, ...]) -> bool:
        """
        Determine if chunk2 can follow chunk1 in a sequence.
        Either through overlap or adjacency.
        """
        # Check if there's an overlap
        for overlap_size in range(1, min(len(chunk1), len(chunk2))):
            if chunk1[-overlap_size:] == chunk2[:overlap_size]:
                return True
        
        # Check if there's an edge from the last element of chunk1 to the first of chunk2
        last_of_chunk1 = chunk1[-1]
        first_of_chunk2 = chunk2[0]
        
        return self.graph.has_edge(last_of_chunk1, first_of_chunk2)

    def segment(self, pos_sequence: List[str]) -> List[List[str]]:
        """
        Segment a POS sequence into chunks based on boundary probabilities.
        
        Args:
            pos_sequence: List of POS tags for a sentence
            
        Returns:
            List of chunks, where each chunk is a list of POS tags
        """
        chunks = []
        current_chunk = [pos_sequence[0]]
        
        for i in range(1, len(pos_sequence)):
            pos1, pos2 = pos_sequence[i-1], pos_sequence[i]
            
            # Get boundary probability
            boundary_prob = self.boundary_probs.get((pos1, pos2), 0.2)  # Default if unseen
            
            if boundary_prob > self.hard_boundary_threshold:
                # Hard boundary - create a new chunk
                chunks.append(current_chunk)
                current_chunk = [pos2]
            else:
                # Continue current chunk
                current_chunk.append(pos2)
        
        # Add the last chunk
        if current_chunk:
            chunks.append(current_chunk)
        
        return chunks

    def predict_next_pos(self, context: List[str], top_n: int = 3) -> List[Tuple[str, float]]:
        """
        Predict the next POS tag given a context sequence.
        
        Args:
            context: List of preceding POS tags
            top_n: Number of top predictions to return
            
        Returns:
            List of (pos_tag, probability) pairs, sorted by probability
        """
        if not context:
            # No context, use connections from start node
            predictions = []
            for target, data in self.graph.out_edges("<START>", data=True):
                predictions.append((target, data["weight"]))
            return sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]
        
        # Use the last tag for prediction
        last_pos = context[-1]
        
        if self.graph.has_node(last_pos):
            # Get all outgoing edges
            predictions = []
            for _, target, data in self.graph.out_edges(last_pos, data=True):
                if target != "<END>":  # Skip end node in predictions
                    predictions.append((target, data["weight"]))
            
            return sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]
        else:
            # Unseen POS tag
            return [("<UNK>", 1.0)]  # Return unknown with full probability

    def recognize_chunks(self, pos_sequence: List[str]) -> List[Dict[str, Any]]:
        """
        Recognize known chunks in a POS sequence.
        
        Args:
            pos_sequence: List of POS tags
            
        Returns:
            List of recognized chunks with their properties
        """
        recognized = []
        
        # Try to match chunks of different sizes
        for i in range(len(pos_sequence)):
            for size in range(4, 1, -1):  # Try larger chunks first (4, 3, 2)
                if i + size <= len(pos_sequence):
                    chunk_tuple = tuple(pos_sequence[i:i+size])
                    if chunk_tuple in self.common_chunks:
                        recognized.append({
                            "chunk": self.common_chunks[chunk_tuple],
                            "start": i,
                            "end": i + size,
                            "activation": self.common_chunks[chunk_tuple]["cohesion"]
                        })
        
        # Sort by start position
        recognized.sort(key=lambda x: x["start"])
        
        return recognized

    def predictive_processing(self, pos_sequence: List[str]) -> Tuple[List[Dict[str, Any]], List[List[str]]]:
        """
        Process a sequence using predictive coding principles.
        
        Args:
            pos_sequence: List of POS tags
            
        Returns:
            Tuple of (recognized chunks, segmented sequence)
        """
        # First pass: recognize chunks
        recognized_chunks = self.recognize_chunks(pos_sequence)
        
        # Second pass: resolve overlaps and calculate activation
        non_overlapping = self._resolve_chunk_overlaps(recognized_chunks, len(pos_sequence))
        
        # Third pass: final segmentation based on chunks and boundaries
        segmentation = self._create_final_segmentation(pos_sequence, non_overlapping)
        
        return non_overlapping, segmentation

    def _resolve_chunk_overlaps(self, chunks: List[Dict[str, Any]], seq_length: int) -> List[Dict[str, Any]]:
        """
        Resolve overlapping chunks by selecting the most activated ones.
        
        Args:
            chunks: List of recognized chunks
            seq_length: Length of the original sequence
            
        Returns:
            List of non-overlapping chunks
        """
        # If no chunks, return empty list
        if not chunks:
            return []
            
        # Sort by activation (cohesion) to prioritize strongest chunks
        sorted_chunks = sorted(chunks, key=lambda x: x["activation"], reverse=True)
        
        # Track which positions are covered
        covered = [False] * seq_length
        
        # Select non-overlapping chunks
        selected = []
        
        for chunk in sorted_chunks:
            start, end = chunk["start"], chunk["end"]
            
            # Check if this chunk overlaps with already selected ones
            overlap = False
            for i in range(start, end):
                if covered[i]:
                    overlap = True
                    break
            
            if not overlap:
                # Add chunk and mark positions as covered
                selected.append(chunk)
                for i in range(start, end):
                    covered[i] = True
        
        # Sort by start position
        selected.sort(key=lambda x: x["start"])
        
        return selected

    def _create_final_segmentation(self, pos_sequence: List[str], chunks: List[Dict[str, Any]]) -> List[List[str]]:
        """
        Create final segmentation based on recognized chunks and boundary probabilities.
        
        Args:
            pos_sequence: Original POS sequence
            chunks: Non-overlapping chunks
            
        Returns:
            List of segments (chunks)
        """
        # If no chunks recognized, fall back to boundary-based segmentation
        if not chunks:
            return self.segment(pos_sequence)
        
        # Create segmentation based on recognized chunks and boundaries
        segmentation = []
        current_pos = 0
        
        for chunk in chunks:
            start, end = chunk["start"], chunk["end"]
            
            # If there's a gap before this chunk, segment it using boundaries
            if start > current_pos:
                gap_sequence = pos_sequence[current_pos:start]
                gap_segments = self.segment(gap_sequence)
                segmentation.extend(gap_segments)
            
            # Add the recognized chunk
            segmentation.append(pos_sequence[start:end])
            current_pos = end
        
        # Handle any remaining sequence after the last chunk
        if current_pos < len(pos_sequence):
            remaining = pos_sequence[current_pos:]
            remaining_segments = self.segment(remaining)
            segmentation.extend(remaining_segments)
        
        return segmentation

    def visualize_pos_graph(self, filename: str = "pos_graph.png"):
        """
        Visualize the POS transition graph.
        
        Args:
            filename: Output file name
        """
        # Check if graph is empty
        if len(self.graph) <= 2:  # Only START and END nodes
            print("POS graph is empty or contains only special nodes - no visualization created")
            return
            
        # Create a copy without special nodes for cleaner visualization
        g = self.graph.copy()
        
        # Only remove special nodes if they exist
        if "<START>" in g:
            g.remove_node("<START>")
        if "<END>" in g:
            g.remove_node("<END>")
            
        if len(g.edges()) == 0:
            print("POS graph has no edges - no visualization created")
            return
        
        # Set up the plot
        plt.figure(figsize=(12, 10))
        
        # Define node positions using spring layout
        pos = nx.spring_layout(g, seed=42)
        
        # Draw nodes
        nx.draw_networkx_nodes(g, pos, node_size=500, node_color="lightblue")
        
        # Prepare edge attributes
        edge_width = []
        edge_color = []
        
        for _, _, data in g.edges(data=True):
            # Default to 0.1 if weight is missing or zero
            weight = data.get("weight", 0.1)
            if weight == 0:
                weight = 0.1
            edge_width.append(weight * 5)
            
            # Default to 0.5 if boundary_prob is missing
            edge_color.append(data.get("boundary_prob", 0.5))
        
        # Draw edges
        nx.draw_networkx_edges(
            g, pos, width=edge_width, 
            edge_color=edge_color, edge_cmap=plt.cm.Reds,
            connectionstyle="arc3,rad=0.1"
        )
        
        # Add labels
        nx.draw_networkx_labels(g, pos, font_size=10)
        
        # Edge labels (probabilities)
        edge_labels = {}
        for u, v, d in g.edges(data=True):
            if "weight" in d:
                edge_labels[(u, v)] = f"{d['weight']:.2f}"
            else:
                edge_labels[(u, v)] = "0.00"
                
        nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels, font_size=8)
        
        # Add a color bar for boundary probabilities
        fig = plt.gcf()
        ax = plt.gca()
        sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds)
        sm.set_array([])
        fig.colorbar(sm, ax=ax, label="Boundary Probability")
        
        plt.title("POS Transition Graph")
        plt.axis("off")
        plt.tight_layout()
        plt.savefig(filename)
        print(f"POS graph visualization saved to {filename}")
        plt.close()

    def visualize_chunk_graph(self, filename: str = "chunk_graph.png"):
        """
        Visualize the chunk transition graph.
        
        Args:
            filename: Output file name
        """
        if len(self.chunk_graph) == 0:
            print("Chunk graph is empty - no visualization created")
            return
            
        if len(self.chunk_graph.edges()) == 0:
            print("Chunk graph has no edges - adding artificial edges for visualization")
            # Create some artificial edges just for visualization
            nodes = list(self.chunk_graph.nodes())
            if len(nodes) > 1:
                for i in range(len(nodes)-1):
                    self.chunk_graph.add_edge(nodes[i], nodes[i+1], weight=0.1)
            
        # Set up the plot
        plt.figure(figsize=(14, 12))
        
        # Define node positions using spring layout
        pos = nx.spring_layout(self.chunk_graph, seed=42)
        
        # Draw nodes with size proportional to cohesion
        node_sizes = []
        for node in self.chunk_graph.nodes():
            cohesion = self.chunk_graph.nodes[node].get("cohesion", 0.5)
            if cohesion <= 0:
                cohesion = 0.5  # Ensure minimum size
            node_sizes.append(cohesion * 800)
        
        nx.draw_networkx_nodes(
            self.chunk_graph, pos, 
            node_size=node_sizes, 
            node_color="lightgreen"
        )
        
        # Draw edges with width proportional to weight
        if len(self.chunk_graph.edges()) > 0:
            edge_width = []
            for _, _, data in self.chunk_graph.edges(data=True):
                weight = data.get("weight", 0.1)
                if weight <= 0:
                    weight = 0.1  # Ensure minimum width
                edge_width.append(weight * 8)
            
            nx.draw_networkx_edges(
                self.chunk_graph, pos, width=edge_width, 
                edge_color="gray", alpha=0.6,
                connectionstyle="arc3,rad=0.1"
            )
        
        # Add labels
        nx.draw_networkx_labels(self.chunk_graph, pos, font_size=9)
        
        plt.title("Chunk Transition Graph")
        plt.axis("off")
        plt.tight_layout()
        plt.savefig(filename)
        print(f"Chunk graph visualization saved to {filename}")
        plt.close()


# Example usage
if __name__ == "__main__":
    # Sample POS sequences for training - Create more examples with repeating patterns
    # to increase likelihood of chunk detection
    training_data = [
        ["DET", "ADJ", "NOUN", "VERB", "DET", "NOUN"],
        ["PRON", "VERB", "PREP", "DET", "NOUN"],
        ["DET", "NOUN", "VERB", "ADV", "ADJ"],
        ["DET", "ADJ", "ADJ", "NOUN", "VERB", "PREP", "DET", "NOUN"],
        ["PRON", "VERB", "DET", "NOUN", "CONJ", "VERB", "ADV"],
        # More examples with repeating patterns to help chunk detection
        ["DET", "ADJ", "NOUN", "VERB", "PREP", "DET", "NOUN"],  # Repeat
        ["DET", "ADJ", "NOUN", "VERB", "DET", "NOUN"],          # Repeat
        ["DET", "NOUN", "VERB", "PREP", "DET", "ADJ", "NOUN"],
        ["PRON", "VERB", "ADV", "CONJ", "VERB", "DET", "NOUN"],
        ["DET", "ADJ", "NOUN", "VERB", "ADV", "PREP", "PRON"],
        ["NOUN", "VERB", "DET", "ADJ", "NOUN", "PREP", "DET", "NOUN"],
        ["DET", "NOUN", "VERB", "ADJ", "CONJ", "ADV"],
        # Even more repetition of common patterns
        ["DET", "ADJ", "NOUN", "VERB", "DET", "NOUN"],          # Repeat
        ["PRON", "VERB", "PREP", "DET", "NOUN"],                # Repeat
        ["DET", "ADJ", "NOUN", "VERB", "PREP", "DET", "NOUN"],  # Repeat
    ]
    
    print(f"Training on {len(training_data)} sentences")
    
    # Initialize and train the graph
    pos_graph = POSGraph()
    pos_graph.train(training_data)
    
    # Test on a new sentence
    test_sentence = ["DET", "ADJ", "NOUN", "VERB", "PREP", "DET", "NOUN"]
    
    print("\nTest sentence:", test_sentence)
    
    # Recognition and segmentation
    chunks, segments = pos_graph.predictive_processing(test_sentence)
    
    print("\nRecognized chunks:")
    if chunks:
        for chunk in chunks:
            print(f"  {chunk['chunk']['elements']} (Position {chunk['start']}-{chunk['end']})")
    else:
        print("  No chunks recognized")
    
    print("\nFinal segmentation:", segments)
    
    # Predictions
    context = ["DET", "ADJ"]
    predictions = pos_graph.predict_next_pos(context)
    print(f"\nTop predictions after {context}:")
    for pos, prob in predictions:
        print(f"  {pos}: {prob:.2f}")
    
    # Visualize graphs
    pos_graph.visualize_pos_graph()
    pos_graph.visualize_chunk_graph()

Training on 15 sentences
Found 83 potential chunks
27 chunks met frequency criteria, 6 met cohesion criteria
Training complete on 15 sentences.
POS graph has 10 nodes and 27 edges
Chunk graph has 6 nodes and 19 edges

Test sentence: ['DET', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 'NOUN']

Recognized chunks:
  ('ADJ', 'NOUN') (Position 1-3)
  ('PREP', 'DET') (Position 4-6)

Final segmentation: [['DET'], ['ADJ', 'NOUN'], ['VERB'], ['PREP', 'DET'], ['NOUN']]

Top predictions after ['DET', 'ADJ']:
  NOUN: 0.75
  ADJ: 0.08
  CONJ: 0.08
POS graph visualization saved to pos_graph.png
Chunk graph visualization saved to chunk_graph.png
